In [11]:
import os
import numpy as np
import pandas as pd
from google.colab import drive
#import ktrain
#from ktrain import text
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
import tensorflow as tf
from tensorflow.python.keras.layers import Dense, Flatten, Input, LSTM, Conv1D, GRU, Dropout
from tensorflow.python.keras.layers.embeddings import Embedding
from tensorflow.python.keras.preprocessing.text import one_hot
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.models import Sequential, Model, load_model
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.python.keras import regularizers
import string
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
drive.mount('/content/drive/')
print(os.listdir('./'))
os.chdir('drive/My Drive/SWM-stock-prediction/code')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
['.ipynb_checkpoints', 'articlescraper.py', 'metadata.csv', 'news_join.ipynb', 'preprocess.ipynb', 'stockdata', 'tweetdata', 'unzip_data.py', 'model_bertBCE.h5', 'model_bert.ckpt.data-00001-of-00002', 'model_bert.ckpt.data-00000-of-00002', 'model_bert.ckpt.index', 'checkpoint', 'bert_Sentence_training.png']


FileNotFoundError: ignored

In [0]:
def split_document_to_chunks(row, field_name, chunk_len):
    text = row[field_name]
    if(text!='[]'):
        label = row['label']
        sentences = sent_tokenize(text)
        output = []
        for i in range(0,len(sentences), chunk_len):
            if(i+chunk_len < len(sentences)):
                chunk = ''.join(sentences[i:i+chunk_len])
            else:
                chunk = ''.join(sentences[i:len(sentences)])
            output.append((chunk,label))
        return output
def prepare_data_set(data_set):
    chunked_text_labels = data_set.apply(split_document_to_chunks, args=('filteredtext_amzn', 3), axis=1)
    X=[]
    y=[]
    for chunks in chunked_text_labels:
        if(chunks is not None):
            for chunk in chunks:
                X.append(chunk[0])
                y.append(chunk[1])
    return X,y

**Loading the dataset**

In [0]:
X_train = np.load('../Data/X_train_bert.npy')
X_test = np.load('../Data/X_test_bert.npy')
train_label = np.load('../Data/label_train_bert.npy')
test_label = np.load('../Data/label_test_bert.npy')

**Train and test shapes**

In [15]:
print(X_train.shape)
print(X_test.shape)

(59278, 768)
(13349, 768)


**Loading the trained BERT Sentence embedding + DNN Model**

In [0]:
model = load_model('../Data/bert_sentence_NN.h5')

**DNN Architecture**

In [17]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 768)               590592    
_________________________________________________________________
dropout_7 (Dropout)          (None, 768)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 769       
Total params: 591,361
Trainable params: 591,361
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.evaluate(X_test, test_label)

418/418 [==============================] - 1s 2ms/step - loss: 0.6531 - accuracy: 0.6383


[0.6531215310096741, 0.6383249759674072]

In [0]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, roc_auc_score

In [20]:
ypred = model.predict_classes(X_test)
print(confusion_matrix(test_label, ypred))
print(precision_score(test_label, ypred))
print(recall_score(test_label, ypred))
print(roc_auc_score(test_label, ypred))


Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
[[4130 2722]
 [2106 4391]]
0.6173203992689442
0.675850392488841
0.6392970584744264
